In [1]:
from toolbox import *
from music21 import *

In [2]:
file = converter.parse('stalker.mid')


In [3]:
file.parts[2].measure(52)
file.measure(53).parts[2]

<music21.stream.Part 0x7f390eaa82b0>

In [4]:
# for track in len(file:
for i in range(len(file.parts)):
    print(len(file.parts[i]))

77
73
51
77
77
43
58
77
73
76
77
77
77
77
77
57
77


In [5]:
for part in file.measure(2):
    print(get_list_entropy(get_pitches(part)))

0.0
0.6365141682948128
0
0
0
0
0
0
0
0
0
0
0.0
0
0
0
0


In [6]:
def get_entropy_from_measure(file,track,measure):
    E_p = get_pitches(file.measure(measure).parts[track])
    E_r = get_rhythm(file.measure(measure).parts[track])
    E_s = get_rests(file.measure(measure).parts[track])
    total_entropy = get_list_entropy(E_p) + get_list_entropy(E_r)+ get_list_entropy(E_s)
    return total_entropy

In [7]:

get_entropy_from_measure(converter.parse('stalker.mid'),1,2)

1.2730283365896256

In [8]:
file.measure(1).parts[2]

<music21.stream.Part 0x7f390e920fa0>

In [11]:
from pyqubo import Binary, Constraint

def max_num_measures(file):
    return max([len(p) for p in file.parts])    

def qubo_form_from(file):
    H = 0
    for j in range(1,max_num_measures(file)):
        c = 0
        o = 0
        for i in range(len(file.parts)):
            if file.parts[i].measure(j) != None:
                c += Binary(f"x_{i}_{j}")
                o += get_entropy_from_measure(file,i,j)*Binary(f"x_{i}_{j}")
        H += Constraint((2-c)**2, f"measure_{j}") + o
    return H.compile().to_qubo()

    
# def qubo_form

In [12]:
qubo_form = qubo_form_from(file)

In [14]:
qubo_form

({('x_14_44', 'x_14_44'): -3.0,
  ('x_14_55', 'x_14_55'): -3.0,
  ('x_5_36', 'x_5_36'): -1.7269716634103744,
  ('x_10_50', 'x_10_50'): -1.208240530771945,
  ('x_6_38', 'x_6_38'): -3.0,
  ('x_15_3', 'x_15_3'): -3.0,
  ('x_16_9', 'x_16_9'): -3.0,
  ('x_0_42', 'x_0_42'): -3.0,
  ('x_9_74', 'x_9_74'): -3.0,
  ('x_13_45', 'x_13_45'): -0.22741127776021886,
  ('x_10_73', 'x_10_73'): -0.9053590611739377,
  ('x_13_11', 'x_13_11'): -3.0,
  ('x_4_20', 'x_4_20'): -3.0,
  ('x_15_20', 'x_15_20'): -3.0,
  ('x_3_42', 'x_3_42'): -3.0,
  ('x_10_70', 'x_10_70'): -1.208240530771945,
  ('x_4_59', 'x_4_59'): -3.0,
  ('x_3_53', 'x_3_53'): -3.0,
  ('x_12_24', 'x_12_24'): -3.0,
  ('x_16_10', 'x_16_10'): -3.0,
  ('x_0_51', 'x_0_51'): -3.0,
  ('x_11_68', 'x_11_68'): -1.0383251960594138,
  ('x_4_16', 'x_4_16'): -3.0,
  ('x_14_60', 'x_14_60'): -3.0,
  ('x_1_72', 'x_1_72'): -1.5716842333399559,
  ('x_4_15', 'x_4_15'): -3.0,
  ('x_2_35', 'x_2_35'): -3.0,
  ('x_7_33', 'x_7_33'): -1.1682852656773173,
  ('x_12_72', 'x_

In [ ]:
from pyqubo import Spin
s1, s2, s3, s4 = Spin("s1"), Spin("s2"), Spin("s3"), Spin("s4")
H = (4*s1 + 2*s2 + 7*s3 + s4)**2
model = H.compile()

In [ ]:
qubo, offset = model.to_qubo()

In [ ]:
type(qubo)